In [204]:
using Flows
using ToySystems
using ToySystems.NineModeSystemEq
using Optim
using PyPlot; pygui(true)
include("9msctrl.jl")
import LinearAlgebra: norm
using PyCall
np = pyimport("numpy")
sc = pyimport("scipy")
plt = pyimport("matplotlib.pyplot")

ErrorException: cannot assign a value to variable PyPlot.plt from module Main

In [2]:
dadt = zeros(9)
q    = zeros(9)
dqdt = zeros(9)
store = RAMStorage(zeros(9))
B = [0,0,0,0,1,0,0,0,0];

In [211]:
objfun(u::AbstractVector) = (u[1] - 1)^2 + sum(u[i]^2 for i = 2:9)
function objfun(t, u, dudt, I, dIdt)
    return dIdt[1] = (u[1] - 1)^2 + sum(u[i]^2 for i = 2:9)
end
function objfun(xq::Coupled)
    return objfun(xq[1])
end

function print_factors(x)
   factors = []
   for i in 1:x
        if x % i == 0
            factors = push!(factors, i)
        else
            0 == 0
        end
    end
    return factors
end

function Objective_fun_timestep(n_initial, Re::Real, Tstepmax, magtstep, Tend)
    f = NineModeSystem(Re)
    ts0 = magtstep.*print_factors(Tend)
    ts = ts0[1:searchsortedfirst(ts0,Tstepmax)-1]
    
    u0_list = []
    for i in 1:n_initial
        u0_list = push!(u0_list, rand(9))
    end
    
    Ind_arr = zeros(n_initial)
    output_1 = []
    x1 = []
    average = []
    ts123 = ts
    for i in 1:length(ts)
        ϕ = flow(couple(f,objfun),RK4(couple(zeros(9),zeros(1))), TimeStepConstant(ts[i]))
        output = []
        x2 = []
        
        ti = 0:ts[i]:Tend
        for j in 1:length(u0_list)
            I0 = Float64[0.0]
            u0_local = copy(u0_list[j])
            ϕ(couple(u0_local, I0), extrema(ti))
            output = push!(output, I0[1]/Tend)
            x2 = push!(x2, ts[i])
        end
        x1 = push!(x1, x2)
        output_1 = push!(output_1, output)
        Ind_arr = hcat(Ind_arr, output)
        
        if sort(output)[end]%sort(output)[end] != 0.0
            println("yes")
            sortedz = sort(output).%sort(output)
            new_int = searchsortedlast(sortedz,0.0)
            if new_int == 0
                ts123 = setdiff(ts123, ts[i])
            else
                average = push!(average, sum(sort(output)[1:new_int])/new_int)
            end
        else
            average = push!(average, sum(output)/length(output))
        end
        
        
    end
    
    fig,ax = plt.subplots()
    for i in 1:length(output_1)
        ax.scatter(x1[i],output_1[i], color = "grey", label = "$i", s = 2)
    end
    ax.plot(ts123, average, color = "black", linewidth = 1.0)
    ax.plot(np.unique(ts123), np.poly1d(np.polyfit(ts123, average, 1))(np.unique(ts)), color = "black" ,linewidth = 1.0, linestyle = "dotted")
#     for i in 1:n_initial
#         ax.plot(ts, Ind_arr[i, 2:end],linewidth = 0.1, label = "$i")
#     end
    xlabel(L"Timestep")
    ylabel(L"Objectivefunction")
    return output_1
end

Objective_fun_timestep (generic function with 1 method)

In [212]:
z = Objective_fun_timestep(10, 500, 1.1, 0.01, 10000)

13-element Array{Any,1}:
 Any[0.7678625266626508, 0.7521138766501057, 0.28836576510887124, 0.7304407819475903, 0.7383111142656467, 0.5736400137744808, 0.7112483492306497, 0.7075582882753546, 0.6316528714555609, 0.47877726482801036] 
 Any[0.22045331342737887, 0.7450337433751046, 0.7408028676531528, 0.7496694084544734, 0.16499960178322598, 0.7158390472231109, 0.7355211290785622, 0.3090929117185871, 0.7449921212622954, 0.29977783615049025]
 Any[0.14878529284444966, 0.738660292117991, 0.7098340327563109, 0.7004574183249904, 0.7539467650741475, 0.7143346583896101, 0.5051872302825305, 0.7140495023817898, 0.7137942527138909, 0.12221346575728867]  
 Any[0.7095147389557056, 0.7079684201311743, 0.7230391133591423, 0.4151626249585599, 0.7558655866285345, 0.7249250903564266, 0.6949023543969646, 0.7401812853418737, 0.6951098780179025, 0.6728273602025837]   
 Any[0.7339182429476373, 0.7216592100169782, 0.14781438166512043, 0.7206023739635246, 0.7136485693935187, 0.6922976168674532, 0.723747688273510

In [210]:
fig, ax = plt.subplots()
ax.scatter([1,2,3],[1,2,3],s = 2)

PyObject <matplotlib.collections.PathCollection object at 0x0000000001FC7EB8>

In [183]:
a = zeros(9)
b = [1,2,3,4,5,6,7,8,9]
c = hcat(a,b,b,b)

9×4 Array{Float64,2}:
 0.0  1.0  1.0  1.0
 0.0  2.0  2.0  2.0
 0.0  3.0  3.0  3.0
 0.0  4.0  4.0  4.0
 0.0  5.0  5.0  5.0
 0.0  6.0  6.0  6.0
 0.0  7.0  7.0  7.0
 0.0  8.0  8.0  8.0
 0.0  9.0  9.0  9.0

In [184]:
c[1,2:end]

3-element Array{Float64,1}:
 1.0
 1.0
 1.0